In [32]:
import pickle
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import warnings

# Ignorer les avertissements
warnings.filterwarnings("ignore")

# Charger les données
df = pd.read_csv('2016_Building_Energy_Benchmarking.csv')

# Préparation des données
features = df[['LargestPropertyUseTypeGFA', 'PrimaryPropertyType']].copy()
targets = df[['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']].copy()

# Save the original columns before one-hot encoding
original_columns = features.columns.tolist()

# Transformer les variables catégoriques en variables numériques
features = pd.get_dummies(features, columns=['PrimaryPropertyType'])

# Drop les NaN values du dataset
df = pd.concat([features, targets], axis=1).dropna()
features = df.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'], axis=1)
targets = df[['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']]

# Save the columns of the DataFrame
with open('columns.pkl', 'wb') as f:
    pickle.dump(features.columns.tolist(), f)

# Save the original columns before one-hot encoding
with open('original_columns.pkl', 'wb') as f:
    pickle.dump(original_columns, f)

# Transformation logarithmique de la variable 'LargestPropertyUseTypeGFA'
features['LargestPropertyUseTypeGFA'] = np.log(features['LargestPropertyUseTypeGFA'])

# Rescaling des variables numériques
scaler = StandardScaler()
features[['LargestPropertyUseTypeGFA']] = scaler.fit_transform(features[['LargestPropertyUseTypeGFA']])

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)


# Définir les paramètres pour la recherche de grille
rf_param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

model = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(model, param_distributions=rf_param_grid, n_iter=20, cv=5)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
best_params = random_search.best_params_

predictions = best_model.predict(X_test)
scores = [r2_score(y_test[column], predictions[:, i]) for i, column in enumerate(y_test.columns)]
best_score = np.mean(scores)

# Cross validation score
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
cv_score_mean = np.mean(cv_scores)

print(f"Cross Validation Score: {cv_score_mean}")
print(f"Best Params: {best_params}")
print(f"Best Score: {best_score}")

# Enregistrer le modèle dans un fichier .pkl
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

# Enregistrer le scaler dans un fichier .pkl
joblib.dump(scaler, 'best_scaler.pkl')

Zero or negative values in 'LargestPropertyUseTypeGFA': 0
Some scaled values: 0    0.726587
1    0.664610
2    3.242804
3    0.297358
4    1.117587
Name: LargestPropertyUseTypeGFA, dtype: float64
Cross Validation Score: 0.5258021032620898
Best Params: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 10}
Best Score: 0.7402157485522256


['best_scaler.pkl']

In [31]:
import pickle
import joblib
import pandas as pd
import numpy as np

with open('best_model.pkl', 'rb') as file:
    model = pickle.load(file)

scaler = joblib.load('best_scaler.pkl')

# Load the model's columns
with open('columns.pkl', 'rb') as f:
    model_columns = pickle.load(f)

# Create a new row of data with all zeros
new_data = pd.DataFrame(columns=model_columns)
new_data.loc[0] = [0 for _ in range(len(model_columns))]

# Fill in the data for your prediction
new_data['PrimaryPropertyType_Hotel'] = 1

# The initial value before transformations
initial_value = 88434  # change this to your actual value

# Apply inverse transformations
log_value = np.log(initial_value)
scaled_value = scaler.transform([[log_value]])[0][0]

new_data['LargestPropertyUseTypeGFA'] = scaled_value

predictions = model.predict(new_data)

# Set options for printing
np.set_printoptions(suppress=True, precision=2)

print(predictions)


[[6747639.74     205.25]]


In [40]:
import pickle
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score
import warnings

# Ignorer les avertissements
warnings.filterwarnings("ignore")

# Charger les données
df = pd.read_csv('2016_Building_Energy_Benchmarking.csv')

# Préparation des données
features = df[['LargestPropertyUseTypeGFA', 'PrimaryPropertyType']].copy()
targets = df[['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']].copy()

# Save the original columns before one-hot encoding
original_columns = features.columns.tolist()

# Transformer les variables catégoriques en variables numériques
features = pd.get_dummies(features, columns=['PrimaryPropertyType'])

# Drop les NaN values du dataset
df = pd.concat([features, targets], axis=1).dropna()
features = df.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'], axis=1)
targets = df[['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']]

# Save the columns of the DataFrame
with open('columns.pkl', 'wb') as f:
    pickle.dump(features.columns.tolist(), f)

# Save the original columns before one-hot encoding
with open('original_columns.pkl', 'wb') as f:
    pickle.dump(original_columns, f)

# Check for any zero or negative values in 'LargestPropertyUseTypeGFA'
print("Zero or negative values in 'LargestPropertyUseTypeGFA':", features['LargestPropertyUseTypeGFA'].le(0).sum())

# Transformation logarithmique de la variable 'LargestPropertyUseTypeGFA'
features['LargestPropertyUseTypeGFA'] = np.log(features['LargestPropertyUseTypeGFA'])

# Rescaling des variables numériques
scaler = StandardScaler()
features[['LargestPropertyUseTypeGFA']] = scaler.fit_transform(features[['LargestPropertyUseTypeGFA']])

# Print some scaled values for inspection
print("Some scaled values:", features['LargestPropertyUseTypeGFA'].head())

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Définir les paramètres pour la recherche de grille
et_param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

model = ExtraTreesRegressor(random_state=42)

random_search = RandomizedSearchCV(model, param_distributions=et_param_grid, n_iter=100, cv=5)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
best_params = random_search.best_params_

predictions = best_model.predict(X_test)
scores = [r2_score(y_test[column], predictions[:, i]) for i, column in enumerate(y_test.columns)]
best_score = np.mean(scores)

# Cross validation score
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
cv_score_mean = np.mean(cv_scores)

print(f"Cross Validation Score: {cv_score_mean}")
print(f"Best Params: {best_params}")
print(f"Best Score: {best_score}")

# Enregistrer le modèle dans un fichier .pkl
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

# Enregistrer le scaler dans un fichier .pkl
joblib.dump(scaler, 'best_scaler.pkl')


Zero or negative values in 'LargestPropertyUseTypeGFA': 0
Some scaled values: 0    0.726587
1    0.664610
2    3.242804
3    0.297358
4    1.117587
Name: LargestPropertyUseTypeGFA, dtype: float64
Cross Validation Score: 0.4922524368709686
Best Params: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20}
Best Score: 0.7237203548715406


['best_scaler.pkl']

In [43]:
import pickle
import joblib
import pandas as pd
import numpy as np

with open('best_model.pkl', 'rb') as file:
    model = pickle.load(file)

scaler = joblib.load('best_scaler.pkl')

# Load the model's columns
with open('columns.pkl', 'rb') as f:
    model_columns = pickle.load(f)

# Create a new row of data with all zeros
new_data = pd.DataFrame(columns=model_columns)
new_data.loc[0] = [0 for _ in range(len(model_columns))]

# Fill in the data for your prediction
new_data['PrimaryPropertyType_Hotel'] = 1

# The initial value before transformations
initial_value = 88434  # change this to your actual value

# Apply inverse transformations
log_value = np.log(initial_value)
scaled_value = scaler.transform([[log_value]])[0][0]

new_data['LargestPropertyUseTypeGFA'] = scaled_value

predictions = model.predict(new_data)

# Set options for printing
np.set_printoptions(suppress=True, precision=2)

print(predictions)


[[6370078.45     175.98]]


In [1]:
import pickle
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import warnings

# Ignorer les avertissements
warnings.filterwarnings("ignore")

# Charger les données
df = pd.read_csv('2016_Building_Energy_Benchmarking.csv')

# Préparation des données
features = df[['LargestPropertyUseTypeGFA', 'PrimaryPropertyType']].copy()
targets = df[['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']].copy()

# Save the original columns before one-hot encoding
original_columns = features.columns.tolist()

# Transformer les variables catégoriques en variables numériques
features = pd.get_dummies(features, columns=['PrimaryPropertyType'])

# Drop les NaN values du dataset
df = pd.concat([features, targets], axis=1).dropna()
features = df.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'], axis=1)
targets = df[['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']]

# Save the columns of the DataFrame
with open('columns.pkl', 'wb') as f:
    pickle.dump(features.columns.tolist(), f)

# Save the original columns before one-hot encoding
with open('original_columns.pkl', 'wb') as f:
    pickle.dump(original_columns, f)

# Transformation logarithmique de la variable 'LargestPropertyUseTypeGFA'
features['LargestPropertyUseTypeGFA'] = np.log(features['LargestPropertyUseTypeGFA'])

# Rescaling des variables numériques
scaler = StandardScaler()
features[['LargestPropertyUseTypeGFA']] = scaler.fit_transform(features[['LargestPropertyUseTypeGFA']])

# Diviser les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Définir les paramètres pour la recherche de grille
rf_param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

model = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(model, param_distributions=rf_param_grid, n_iter=20, cv=5)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
best_params = random_search.best_params_

predictions = best_model.predict(X_test)
r2_scores = [r2_score(y_test[column], predictions[:, i]) for i, column in enumerate(y_test.columns)]
rmse_scores = [mean_squared_error(y_test[column], predictions[:, i], squared=False) for i, column in enumerate(y_test.columns)]

# Print R2 scores
print("R2 Scores:")
for column, score in zip(y_test.columns, r2_scores):
    print(f"{column}: {score}")

# Print RMSE scores
print("RMSE Scores:")
for column, score in zip(y_test.columns, rmse_scores):
    print(f"{column}: {score}")

# Cross validation score
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5)
cv_score_mean = np.mean(cv_scores)

print(f"Cross Validation Score: {cv_score_mean}")
print(f"Best Params: {best_params}")

# Enregistrer le modèle dans un fichier .pkl
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

# Enregistrer le scaler dans un fichier .pkl
joblib.dump(scaler, 'best_scaler.pkl')


R2 Scores:
SiteEnergyUse(kBtu): 0.6284824756974252
TotalGHGEmissions: 0.8867741478990165
RMSE Scores:
SiteEnergyUse(kBtu): 12351625.219564518
TotalGHGEmissions: 192.43179331187875
Cross Validation Score: 0.5237068864600779
Best Params: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 40}


['best_scaler.pkl']

In [8]:
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import ShuffleSplit, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score, mean_squared_error
import warnings

warnings.filterwarnings("ignore")

df = pd.read_csv('C:/Users/loren/OneDrive/Documents/GitHub/co2/2016_Building_Energy_Benchmarking.csv')

X = df[['LargestPropertyUseTypeGFA', 'PrimaryPropertyType']].copy()
y = df[['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']].copy()
X[['LargestPropertyUseTypeGFA']] += 0.01

df = pd.concat([X, y], axis=1).dropna()


X = df.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'], axis=1)
# Utiliser une constante plus grande pour garantir que toutes les valeurs soient positives avant la transformation logarithmique
y = np.log(df[['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']] + 1) 

log_transformer = FunctionTransformer(np.log, validate=True)
one_hot_encoder = OneHotEncoder(sparse=False, drop='first')
scaler = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('log', log_transformer, ['LargestPropertyUseTypeGFA']),
        ('one_hot', one_hot_encoder, ['PrimaryPropertyType'])
    ])

models = [
    ("Extra Trees", ExtraTreesRegressor(), {'model__estimator__n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500],
                                            'model__estimator__max_depth': [None, 5, 10, 15, 20, 25, 30]})
]

best_model = None
best_scaler = None
best_avg_test_score = -np.inf 

results = []
n_splits = 5
test_size = 0.2
shuffle_split = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)

for name, model, params in models:
    for train_index, test_index in shuffle_split.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                                   ('scaler', scaler), 
                                   ('model', MultiOutputRegressor(model))])

        random_search = RandomizedSearchCV(pipeline, param_distributions=params, n_iter=10, cv=5, random_state=42)
        random_search.fit(X_train, y_train)
        best_model = random_search.best_estimator_

        y_train_pred = best_model.predict(X_train)
        y_test_pred = best_model.predict(X_test)
        
        y_train_pred_exp = np.exp(y_train_pred) - 0.01
        y_test_pred_exp = np.exp(y_test_pred) - 0.01
        y_train_exp = np.exp(y_train) - 0.01
        y_test_exp = np.exp(y_test) - 0.01

        for i, target in enumerate(y.columns):
            r2_train = r2_score(y_train_exp.iloc[:, i], y_train_pred_exp[:, i])
            r2_test = r2_score(y_test_exp.iloc[:, i], y_test_pred_exp[:, i])
            rmse_train = mean_squared_error(y_train_exp.iloc[:, i], y_train_pred_exp[:, i], squared=False)
            rmse_test = mean_squared_error(y_test_exp.iloc[:, i], y_test_pred_exp[:, i], squared=False)

            results.append([name, target, r2_train, rmse_train, r2_test, rmse_test])

        avg_test_score = np.mean([result[-2] for result in results[-2:]])

        if avg_test_score > best_avg_test_score:
            best_avg_test_score = avg_test_score
            best_model = random_search.best_estimator_
            best_scaler = scaler

results_df = pd.DataFrame(results, columns=['Model', 'Target', 'R2 Score (Training)', 'RMSE (Training)', 'R2 Score (Testing)', 'RMSE (Testing)'])

print(f"Best model average R2 Score (Testing): {best_avg_test_score}")
print(results_df)
print(f"\nBest model: {best_model}")

joblib.dump(best_model, 'logbest_model.pkl')
joblib.dump(best_scaler, 'logbest_scaler.pkl')


Best model average R2 Score (Testing): 0.7720214034846744
         Model               Target  R2 Score (Training)  RMSE (Training)  \
0  Extra Trees  SiteEnergyUse(kBtu)             0.997224     1.158422e+06   
1  Extra Trees    TotalGHGEmissions             0.993794     4.184152e+01   
2  Extra Trees  SiteEnergyUse(kBtu)             0.166481     2.092170e+07   
3  Extra Trees    TotalGHGEmissions             0.325687     4.384207e+02   
4  Extra Trees  SiteEnergyUse(kBtu)             0.586319     9.116150e+06   
5  Extra Trees    TotalGHGEmissions             0.768881     2.092103e+02   
6  Extra Trees  SiteEnergyUse(kBtu)             0.978454     3.166246e+06   
7  Extra Trees    TotalGHGEmissions             0.992066     4.064510e+01   
8  Extra Trees  SiteEnergyUse(kBtu)             0.365620     1.280530e+07   
9  Extra Trees    TotalGHGEmissions             0.375098     4.086844e+02   

   R2 Score (Testing)  RMSE (Testing)  
0            0.645341    1.206812e+07  
1            0

['logbest_scaler.pkl']